# Task 2a
Die folgenden Kapitel widmen sich der Datenvorbereitung der gecrawlten Textabschnitte mit Identifizierung von Named Entities und nicht-NE Touristenattraktionen sowie einem anschließenden Clustering der Dokumente nach den nicht-NE Touristenattraktionen.

----------------------------------

#### Inhaltsverzeichnis:
#### 1. [Text Preprocessing](#Text_prep)
#### 2. [Erstellen von Trainingsdaten](#Build_train_data)
#### 3. [Clustering](#Clustering)
#### 4. [Bonus: Automatisches mapping der NoNE zu Kategorien](#Bonus)
-----------------------------------------

##### Imports

In [ ]:
from task2a_preprocessing import Preprocessing
from task2a_clustering import Clustering
import pandas as pd

### 1. Text Preprocessing <a class="anchor" id="Text_prep"></a>

In [ ]:
prep_lemma = Preprocessing()
prep_lemma.preprocess_data('results_scrapping.csv', 'Lemmatization')

prep_stem = Preprocessing()
prep_stem.preprocess_data('results_scrapping.csv', 'Stemming')

### 2. Erstellen von Trainingsdaten <a class="anchor" id="Build_train_data"></a>

In [ ]:
grams = ['1-grams','2-grams','mixed-grams']
matrix = ['TFIDF','True/False']
data_list = []

for g in grams:
    for m in matrix:
        data_list.append({'data':prep_lemma.create_train_data(g, m),
                          'wordreduction':'Lemmatization',
                          'grams':g,
                          'matrix':m})
        data_list.append({'data':prep_stem.create_train_data(g, m),
                          'wordreduction':'Stemming',
                          'grams':g,
                          'matrix':m})

### 3. Clustering <a class="anchor" id="Clustering"></a>

In [ ]:
clustering = Clustering()
hyperparameter_dict = {"cluster_types":["KMeans","DBSCAN","AffinityPropagation","AgglomerativeClustering","Birch","BisectingKMeans","MiniBatchKMeans","MeanShift","OPTICS","SpectralClustering"],
                       "n_clusters":[2,3,4,5,6,7,8,9,10]}
df_results = clustering.run_clustering(data_list, hyperparameter_dict)

In [ ]:
df_results.to_csv('results_2a.csv')

In [ ]:
df_mapping_lemma = prep_lemma.df_mapping
df_mapping_stem = prep_stem.df_mapping

In [ ]:
df_mapping_lemma.to_csv('mapping_data_lemma.csv')
df_mapping_stem.to_csv('mapping_data_stem.csv')